In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
plt.style.use('fivethirtyeight')
from scipy import stats
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
missing = train.isna().sum()
missing = missing[missing>0]
missing_perc = missing/train.shape[0]*100
na=pd.DataFrame([missing,missing_perc],index = ['missing_num','missing_perc']).T
na=na.sort_values(by = 'missing_perc',ascending=False)


In [ ]:
sns.pairplot(x_vars = ['PoolQC','MiscFeature','Alley','Fence'],y_vars = 'SalePrice',data = train)
plt.show()
sns.pairplot(x_vars = ['FireplaceQu','GarageType','GarageYrBlt','GarageFinish'],y_vars = 'SalePrice',data = train)
plt.show()
sns.pairplot(x_vars = ['BsmtExposure','BsmtFinType2','BsmtFinType1','MasVnrType'],y_vars = 'SalePrice',data = train)
plt.show()
plt.savefig('myfig.jpg')

In [2]:
#Handle Missing Values
train.drop(['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageType','GarageYrBlt','GarageFinish','BsmtExposure','BsmtFinType2','BsmtFinType1','MasVnrType'],axis=1,inplace = True)

NameError: name 'train' is not defined

In [ ]:
train.drop(train[train.Electrical.isna()].index,axis=0,inplace=True)
NA = ['GarageQual','GarageCond','BsmtCond','BsmtQual']
for na in NA:
    train[na].fillna('NA',inplace=True)
train['MasVnrArea'].fillna(0,inplace=True)
train['LotFrontage'].fillna(0,inplace=True)

In [ ]:
sns.distplot(train.SalePrice)
plt.show()

In [ ]:
figure=plt.figure(figsize=(13,5))
plt.subplot(1,2,1)
stats.probplot(train.SalePrice, plot=plt)
plt.title('Actual SalePrice')
plt.subplot(1,2,2)
train.SalePrice=np.log(train.SalePrice)
stats.probplot(train.SalePrice, plot=plt)
plt.title('SalePrice after log transformation')
plt.show()
plt.savefig('SalePrice.jpg')
plt.figure(figsize=(15,10))
sns.heatmap(train.corr(),vmax=8,square=True)
plt.savefig('Heatmap.jpg')

In [ ]:
plt.figure(figsize=(11,4))
plt.subplot(1,2,1)
plt.scatter(train.OverallQual,train.SalePrice)
plt.xlabel('OverallQual');plt.ylabel('SalePrice')
plt.subplot(1,2,2)
plt.scatter(train.GrLivArea,train.SalePrice)
plt.xlabel('Grnd Living Area'); plt.ylabel('SalePrice')
plt.show()

In [ ]:
var=['GarageCars','GarageArea','TotalBsmtSF','1stFlrSF','FullBath']
sns.pairplot(x_vars=var,y_vars='SalePrice',data=train)
plt.show()

In [ ]:
train.drop('Id',axis=1,inplace=True)
train.drop(['TotRmsAbvGrd','GarageArea'],axis=1,inplace=True)
features=['OverallQual','GrLivArea']
for feat in features:
    train[feat+'_p2'] = train[feat]**2
    train[feat+'_p3'] = train[feat]**3
mp = {'Ex': 5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}
for feat in ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual','GarageQual','GarageCond']:
    train[feat] = train[feat].map(mp)
mp={'N':0,'Y':2,'P':1}
for feat in ['CentralAir','PavedDrive']:
    train[feat]=train[feat].map(mp)
mp = {'Typ':8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4,'Maj2':3,'Sev':2,'Sal':1}
train['Functional']=train['Functional'].map(mp)
mp={'Gtl':1,'Mod':2,'Sev':3}

In [ ]:
train['LandSlope']=train['LandSlope'].map(mp)
train['TotBath']=train['BsmtFullBath']+train['FullBath']+.5*(train.BsmtHalfBath+train.HalfBath)
train['Overall_Score']=train.OverallQual*train.OverallCond
train['Total_area']=train['1stFlrSF']+train['2ndFlrSF']+train.TotalBsmtSF
train['Garage_Score']=train.GarageQual*train.GarageCond
train['Kitchen_Score']=train.KitchenAbvGr*train.KitchenQual
train['Bsmt_Score']=train.BsmtQual*train.BsmtCond

In [ ]:
train=pd.get_dummies(train)

In [ ]:
train_x = train.drop('SalePrice',axis=1)
train_y = pd.DataFrame(train.SalePrice)
index = train_x.columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
train_x = pd.DataFrame(train_x,columns=index)

In [ ]:
#Test Data
test.drop(['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageType','GarageYrBlt','GarageFinish','BsmtExposure','BsmtFinType2','BsmtFinType1','MasVnrType'],axis=1,inplace = True)

In [ ]:
test.drop(test[test.Electrical.isna()].index,axis=0,inplace=True)
NA = ['GarageQual','GarageCond','BsmtCond','BsmtQual']
for na in NA:
    test[na].fillna('NA',inplace=True)
    NA = ['GarageQual','GarageCond','BsmtCond','BsmtQual']
for na in NA:
    test[na].fillna('NA',inplace=True)
fill_zeros = ['GarageCars','GarageArea']
for zeros in fill_zeros:
    test[zeros].fillna(0,inplace=True)
test['MasVnrArea'].fillna(0,inplace=True)
test['LotFrontage'].fillna(0,inplace=True)
test['Electrical'].fillna('SBrkr',inplace=True)
test.MSZoning.fillna('RL',inplace=True)
test.Utilities.fillna('AllPub',inplace=True)
test.BsmtFinSF1.fillna(0.0,inplace=True)
test.BsmtFinSF2.fillna(0.0,inplace=True)
test.BsmtUnfSF.fillna(0.0,inplace=True)
test.TotalBsmtSF.fillna(0.0,inplace=True)
test.BsmtFullBath.fillna(0.0,inplace=True)
test.BsmtHalfBath.fillna(0.0,inplace=True)
test.Functional.fillna('Typ',inplace=True)
test.Exterior1st.fillna('VinylSd',inplace=True)
test.Exterior2nd.fillna('VinylSd',inplace=True)
test.KitchenQual.fillna('TA',inplace=True)
test.SaleType.fillna('WD',inplace=True)
test.drop('Id',axis=1,inplace=True)
features = ['OverallQual','GrLivArea']
for feat in features:
    test[feat+'_p2'] = test[feat]**2
    test[feat+'_p3'] = test[feat]**3
test.drop(['TotRmsAbvGrd','GarageArea'],axis=1,inplace=True)
mp = {'Ex': 5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}
for feat in ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual','GarageQual','GarageCond']:
    test[feat] = test[feat].map(mp)
mp={'N':0,'Y':2,'P':1}
for feat in ['CentralAir','PavedDrive']:
    test[feat]=test[feat].map(mp)
mp = {'Typ':8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4,'Maj2':3,'Sev':2,'Sal':1}
test['Functional']=test['Functional'].map(mp)
mp={'Gtl':1,'Mod':2,'Sev':3}

In [ ]:
test['LandSlope']=test['LandSlope'].map(mp)
test['TotBath']=test['BsmtFullBath']+test['FullBath']+.5*(test.BsmtHalfBath+test.HalfBath)
test['Overall_Score']=test.OverallQual*test.OverallCond
test['Total_area']=test['1stFlrSF']+test['2ndFlrSF']+test.TotalBsmtSF
test['Garage_Score']=test.GarageQual*test.GarageCond
test['Kitchen_Score']=test.KitchenAbvGr*test.KitchenQual
test['Bsmt_Score']=test.BsmtQual*test.BsmtCond
test=pd.get_dummies(test)

In [ ]:
testcol = test.columns.tolist()
traincol = train_x.columns.tolist()
diff = list(set(traincol).difference(testcol))
last_cols = train_x[diff]
train_x.drop(diff,axis=1,inplace=True)
index = test.columns
scaler = StandardScaler()
test = scaler.fit_transform(test)
test = pd.DataFrame(test,columns = index)


In [ ]:
#Model Creation
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import mean_squared_error as mse
model1 = Ridge(alpha=1)
model2 = LinearRegression()
model2.fit(train_x,train_y)
model1.fit(train_x,train_y)
print("The accuracy of training LR set is",model2.score(train_x,train_y))
predRidge= model1.predict(test)
predLR=model2.predict(test)
print("The accuracy of training Ridge set is",model1.score(train_x,train_y))
